# Add and search by job tags

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.40.1
```
</details>

This guide focuses on how to add and update job tags, as well as how to search by job tags, so that you can better organize, track, and understand your experiments.

## Add and search by job tags

### Assign tags

You can assign one or more tags to your jobs when you run them so that you can filter by a tag later. You might want to use job tags to label particular error mitigation settings, circuit parameters, and so forth.

In [19]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = Sampler(backend)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

# Assign tags before executing
sampler.options.environment.job_tags = ["experiment-2025", "sampler-example"]

# Submit
job = sampler.run([isa_circuit])

print(service.job(job.job_id()).tags)

['experiment-2025', 'sampler-example']


### Add and update tags

You can add tags after you submit a job with the `update_tags()` method. This method overwrites current tags, so if you have already assigned tags to a job and you want to add additional tags, be sure to re-assign the original tags.

In [20]:
# Add a new tag while keeping the previously assigned tags
job.update_tags(["experiment-2025", "sampler-example", "127-qubit"])

# Confirm updated tags
print(job.tags)

['sampler-example', 'experiment-2025', '127-qubit']


### Retrieve jobs by tag

Display a list of jobs with certain tags as follows:

In [21]:
# List jobs with a specific tag or set of tags

service.jobs(job_tags=["experiment-2025", "sampler-example"])

[<RuntimeJobV2('d38l3pkgenls73cf263g', 'sampler')>,
 <RuntimeJobV2('d36s55gitjus73f5li10', 'sampler')>,
 <RuntimeJobV2('d36s4mkda4cs73a8994g', 'sampler')>,
 <RuntimeJobV2('d36rs9o0sqis73982e4g', 'sampler')>,
 <RuntimeJobV2('d36rptsgenls73cda9n0', 'sampler')>,
 <RuntimeJobV2('d36rohkgenls73cda87g', 'sampler')>,
 <RuntimeJobV2('d36rnr0itjus73f5l4m0', 'sampler')>,
 <RuntimeJobV2('d36rnp0itjus73f5l4j0', 'sampler')>,
 <RuntimeJobV2('d35f3joitjus73f49ds0', 'sampler')>,
 <RuntimeJobV2('d35bsn80sqis7396img0', 'sampler')>]